In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/home/mrzaizai2k/code_Bao/ViT5/examples


In [2]:
import sys
sys.path.append("")

In [3]:
# !wget 'https://github.com/ThanhChinhBK/vietnews/archive/master.zip'
# !unzip 'master.zip'

In [4]:
import glob
import pandas as pd
pd.set_option('display.max_columns', None)
import concurrent.futures
from datasets import *
import datasets
from sklearn.model_selection import train_test_split
import transformers
from transformers import RobertaTokenizerFast,AutoTokenizer
from vncorenlp import VnCoreNLP


/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
OUTPUT_DIR = 'my_fine_tuned_t5_small_model'

## Processing data

In [6]:
def listPaths(path):
  pathfiles = list()
  for pathfile in glob.glob(path):
    pathfiles.append(pathfile)
  return pathfiles

train_paths = listPaths('data/vietnews-master/data/train_tokenized/*')
val_paths = listPaths('data/vietnews-master/data/val_tokenized/*')
test_paths = listPaths('data/vietnews-master/data/test_tokenized/*')

In [7]:
def read_content(pathfile):
    """
    Input: Path of txt file
    Output: A dictionary has keys 'original' and 'summary'
    """
    with open(pathfile) as f:
      rows  = f.readlines()
      original = ' '.join(''.join(rows[4:]).split('\n'))
      summary = ' '.join(rows[2].split('\n'))
            
    return {'file' : pathfile,
              'original': original, 
              'summary': summary}

In [8]:
read_content(train_paths[0])

{'file': 'data/vietnews-master/data/train_tokenized/006157.txt.seg',
 'original': 'Tập 4 Thần_tượng Âm_nhạc nhí - Vietnam_Idol_Kids 2017 lên sóng tối 2/6 với màn tranh tài của top 8 thí_sinh nữ để lựa_chọn ra 5 tấm vé vào tiếp vòng trong . Trong đó , 3 giám_khảo Isaac - Văn_Mai_Hương - Bích_Phương được quyền lựa_chọn 3 thí_sinh . 2 thí_sinh còn lại sẽ do các giám_khảo khách mời quyết_định . Nữ ca_sĩ Văn_Mai_Hương xúc_động : “ Cô có may_mắn năm nay ngồi ở vị_trí ban giám_khảo , may_mắn hơn là được gặp con . Mỗi khi gặp Hiền , cô tự thấy bản_thân cô rất kém , bởi có lúc cô không trân_trọng cũng như không tin vào bản_thân ... Cảm_ơn con , bởi đôi_khi có những cái cô không bằng con được , đó là sự lạc_quan . Và cô tin , còn rất nhiều người phải học đức_tính lạc_quan này của Hiền . Con hát rất là hay ” . Đồng_tình với ý_kiến của đồng_nghiệp , Bích_Phương cũng xúc_động chia_sẻ : “ Giọt nước_mắt dành cho con là sự khâm_phục chứ không phải là thương_cảm . Cô chưa bao_giờ thấy con buồn . Từ lúc

In [9]:
def get_dataframe(pathfiles):
    with concurrent.futures.ProcessPoolExecutor() as executor:
      data = executor.map(read_content, pathfiles)
    
    # Make blank dataframe
    data_df = list()
    for d in data:
      data_df.append(d)
    data_df = pd.DataFrame(data_df)
    data_df.dropna(inplace = True)
    data_df = data_df.sample(frac=1).reset_index(drop=True)

    return data_df

In [10]:
train_df = get_dataframe(train_paths[:100])

In [11]:
val_df = get_dataframe(val_paths[:100])
test_df = get_dataframe(test_paths[:100])

In [12]:
test_df.head()

,file,original,summary
0,data/vietnews-master/data/test_tokenized/02153...,"Ngày 4/4 , hội_nghị đại_biểu Quốc_hội chuyên_t...","Đại_biểu Lưu_Bình_Nhưỡng cho_rằng , cần quy_đị..."
1,data/vietnews-master/data/test_tokenized/00352...,Dự_án cải_tạo Bến xe_buýt Tân_Quy với kinh_phí...,Trung_tâm Quản_lý giao_thông công_cộng TP. HCM...
2,data/vietnews-master/data/test_tokenized/00323...,"Theo UBND tỉnh Quảng_Ninh , dự_án có tổng mức ...","Chiều 22-8 , UBND tỉnh Quảng_Ninh tổ_chức khởi..."
3,data/vietnews-master/data/test_tokenized/01056...,Tổng_thống Philippines_Rodrigo_Duterte . Bộ Nộ...,Quyết_định mới của chính_quyền Duterte gây ra ...
4,data/vietnews-master/data/test_tokenized/01260...,"Bà Obama giới_thiệu chồng là "" tình_yêu đời tô...","Đệ nhất phu_nhân Mỹ giới_thiệu "" tình_yêu đời ..."


## **Warm-starting RoBERTaShared for BBC XSum**

***Note***: This notebook only uses a few training, validation, and test data samples for demonstration purposes. To fine-tune an encoder-decoder model on the full training data, the user should change the training and data preprocessing parameters accordingly as highlighted by the comments.


### **Data Preprocessing**


In [13]:
pretrained_model = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [14]:
train_data =  Dataset.from_pandas(train_df)
val_data =  Dataset.from_pandas(val_df)
test_data =  Dataset.from_pandas(test_df)

In [15]:
batch_size=16  # change to 16 for full training
encoder_max_length=256
decoder_max_length=64

def process_data_to_model_inputs(batch):                                                               
    # Tokenizer will automatically set [BOS] <text> [EOS]                                               
    model_inputs  = tokenizer(batch["original"], padding="max_length", truncation=True, max_length=encoder_max_length)
    labels = tokenizer(batch["summary"], padding="max_length", truncation=True, max_length=decoder_max_length)
                                                                                                        
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# only use 32 training examples for notebook - DELETE LINE FOR FULL TRAINING
# train_data = train_data.select(range(32))

train_data_batch = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["file","original", "summary"],
)
train_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],
)


# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
# val_data = val_data.select(range(16))

val_data_batch = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["file", "original", "summary"],
)
val_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],
)

Map: 100%|██████████| 100/100 [00:00<00:00, 1543.16 examples/s]


In [16]:
train_df.head(1)

,file,original,summary
0,data/vietnews-master/data/train_tokenized/0739...,Đó là ý_kiến của các chuyên_gia giáo_dục liên_...,"Nếu được nghỉ học thứ_bảy , học_sinh có thêm t..."


In [17]:
train_data_batch[0]

{'input_ids': tensor([  977,   553,   796,   259,   952,   290,   650,  1405,   317,   708,
           262,   259,  1226,   562,  4634,   290, 10745,   259,  2624,   268,
           290,   285,  2902,   615,  1079,   290, 84631,   355,  1492,   290,
         33601,  1075,   970,  2906,   297,   290,   263, 25833,   594,  2802,
          2906,   297,   394,  1978,   290,   316,  1138,   276,   259, 15265,
           290,  7568,   317,  2802,   594,  2802,   562,   708,   290,   272,
         13262,   259,  2180,   382,  1375,   290, 16309,   259,   260,  9448,
           553,   394,  1849,   290,   334,  1375,   259,   272,  3502,  2906,
           297,   333,   758,  1544,   266,   259,   275,   259,  3665,   259,
          3920,   370,   259, 15355,   290,   561,  2296,   290, 26250,  3502,
           259,   264,   690,   553,   259,  2624,   282,   290, 11095,  4059,
           320,  1269, 83438,   264,  8143,   490, 14133,   260,   259, 27398,
           259,   261,   970,   290,   

### **Warm-starting the Encoder-Decoder Model**

In [18]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model)

### **Fine-Tuning Warm-Started Encoder-Decoder Models**

The `Seq2SeqTrainer` that can be found under [examples/seq2seq/seq2seq_trainer.py](https://github.com/huggingface/transformers/blob/master/examples/seq2seq/seq2seq_trainer.py) will be used to fine-tune a warm-started encoder-decoder model.

Let's download the `Seq2SeqTrainer` code and import the module along with `TrainingArguments`.

In [19]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional

2024-06-08 18:46:07.407372: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-08 18:46:07.437992: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-08 18:46:08.062203: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


We need to add some additional parameters to make `TrainingArguments` compatible with the `Seq2SeqTrainer`. Let's just copy the `dataclass` arguments as defined in [this file](https://github.com/patrickvonplaten/transformers/blob/make_seq2seq_trainer_self_contained/examples/seq2seq/finetune_trainer.py).

Also, we need to define a function to correctly compute the ROUGE score during validation. ROUGE is a much better metric to track during training than only language modeling loss.

In [20]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

/tmp/ipykernel_225222/1860282078.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = datasets.load_metric("rouge")
/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Cool! Finally, we start training.

In [23]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir= OUTPUT_DIR,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    # evaluate_during_training=True,
    do_train=True,
    do_eval=True,
    logging_steps=200,  # set to 2000 for full training
    save_steps=5000,  # set to 500 for full training
    eval_steps=7500,  # set to 7500 for full training
    warmup_steps=3000,  # set to 3000 for full training
    num_train_epochs=10, #uncomment for full training
    overwrite_output_dir=True,
    save_total_limit=3,
    fp16=True, 
    push_to_hub=True,
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data_batch,
    eval_dataset=val_data_batch,
)
trainer.train()

Step,Training Loss


TrainOutput(global_step=70, training_loss=0.0, metrics={'train_runtime': 20.6957, 'train_samples_per_second': 48.319, 'train_steps_per_second': 3.382, 'total_flos': 264374845440000.0, 'train_loss': 0.0, 'epoch': 10.0})

In [24]:
trainer.save_model("my_fine_tuned_t5_small_model")

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]









model.safetensors:   0%|          | 16.4k/1.20G [00:00<5:47:05, 57.7kB/s]





events.out.tfevents.1717847220.DESKTOP-H2CRQMR.225222.0: 100%|██████████| 4.18k/4.18k [00:00<00:00, 7.28kB/s]
events.out.tfevents.1717847225.DESKTOP-H2CRQMR.225222.1: 100%|██████████| 5.47k/5.47k [00:00<00:00, 9.64kB/s]
model.safetensors:   0%|          | 2.56M/1.20G [00:00<04:46, 4.18MB/s]  




model.safetensors:   0%|          | 3.44M/1.20G [00:00<04:35, 4.34MB/s]




model.safetensors:   0%|          | 4.49M/1.20G [00:00<03:39, 5.44MB/s]


model.safetensors:   1%|          | 6.64M/1.20G [00:01<03:14, 6.15MB/s]


model.safetensors:   1%|          | 7.91M/1.20G [00:01<02:43, 7.30MB/s]


model.safetensors:   1%|          | 9.39M/1.20G [00:01<02:20, 8.48MB/s]


training_args.bin: 100%|██████████| 5.30k/5.30k [00:00<00:00, 15.8kB/s]
model.safetensors:   1%|          | 10.9M/1.20G [00:01<02:06, 9.41MB/s]


model.safetensors:   1%|         

In [25]:
# !gsutil -m cp -r '/content/training/*' 'gs://kaggle-vbdi-test/training_Data'

### **Evaluation**

Awesome, we finished training our dummy model. Let's now evaluated the model on the test data. We make use of the dataset's handy `.map()` function to generate a summary of each sample of the test data.

In [26]:
import datasets
from transformers import RobertaTokenizer, EncoderDecoderModel, AutoTokenizer, AutoModelForSeq2SeqLM

from sklearn.model_selection import train_test_split

tokenizer = AutoTokenizer.from_pretrained(pretrained_model, use_fast=False)

# model = EncoderDecoderModel.from_pretrained(OUTPUT_DIR + "/checkpoint-4000")
model = AutoModelForSeq2SeqLM.from_pretrained("my_fine_tuned_t5_small_model")

model.to("cuda")

# test_data = datasets.load_dataset("xsum", split="test")

batch_size = 16  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["original"], padding="max_length", truncation=True, max_length=256, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["original"])

pred_str = results["pred"]
label_str = results["summary"]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Map: 100%|██████████| 100/100 [00:01<00:00, 61.07 examples/s]


In [27]:
rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1","rouge2","rougeL"])

In [28]:
for key,value in rouge_output.items():
  print(key)
  print(value.mid)

rouge1
Score(precision=0.14416666666666672, recall=0.015686256903094605, fmeasure=0.02800959396390076)
rouge2
Score(precision=0.017, recall=0.001761819575773064, fmeasure=0.0031891063410072534)
rougeL
Score(precision=0.13818939393939392, recall=0.014820770937357524, fmeasure=0.026534492964907685)


In [29]:
i = 0
print('Prediction: ',pred_str[i])
print('Truth: ',label_str[i])
print('Content: ',test_data[i]['original'])

Prediction:  <extra_id_0>.
Truth:  Đại_biểu Lưu_Bình_Nhưỡng cho_rằng , cần quy_định chặt_chẽ việc cơ_quan nhà_nước tiếp_nhận quà tặng từ doanh_nghiệp . Các tỉnh được doanh_nghiệp tặng ôtô nên đấu_giá để thực_hiện công_tác xã_hội . 
Content:  Ngày 4/4 , hội_nghị đại_biểu Quốc_hội chuyên_trách đã cho ý_kiến vào dự_thảo Luật Quản_lý , sử_dụng tài_sản nhà_nước ( sửa_đổi ) . Ông Lưu_Bình_Nhưỡng , Uỷ_viên thường_trực Uỷ_ban về các vấn_đề xã_hội , nêu vấn_đề thời_gian qua đã xảy ra những sự_việc đáng tiếc liên_quan đến việc các tỉnh , thành nhận ôtô đắt tiền do doanh_nghiệp tặng , sau đó phải trả lại . " Việc quản_lý , sử_dụng các loại tài_sản biếu tặng cho cơ_quan nhà_nước là một bài_học . Do_vậy , dự_thảo luật cần phải quy_định chặt_chẽ " , ông nói . Theo ông Nhưỡng , ranh_giới giữa việc sử_dụng tài_sản là ôtô được tặng cho mục_đích công và nhu_cầu cá_nhân trong trường_hợp này rất khó xác_định , thậm_chí rất dễ nảy_sinh tiêu_cực . Ngoài_ra , việc tỉnh nhận xe của doanh_nghiệp còn có_thể phá

In [30]:
test_data[0]['file']

'data/vietnews-master/data/test_tokenized/021531.txt.seg'